# check generated fiberProfiles quality

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import MultipleLocator
import matplotlib.patches as patches
import scipy.ndimage as nd

from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend("matplotlib")

from pfs.drp.stella.utils import addPfsCursor, showAllSpectraAsImage, showDetectorMap
from pfs.drp.stella import SpectrumSet
from pfs.drp.stella.subtractSky1d import subtractSky1d
from pfs.utils.fiberids import FiberIds
from pfs.datamodel import PfsDesign, FiberStatus, TargetType
from pfs.datamodel import PfsFiberNorms
from pfs.drp.stella.readLineList import ReadLineListTask, ReadLineListConfig
from pfs.drp.stella.subtractSky1d import subtractSky1d
from pfs.datamodel.target import Target

import warnings
warnings.filterwarnings('ignore')

from PIL import Image

plt.rcParams["font.size"] = 12         
plt.rcParams["xtick.direction"] = "in" 
plt.rcParams["ytick.direction"] = "in" 
plt.rcParams["xtick.top"] = True
plt.rcParams["xtick.bottom"] = True    
plt.rcParams["ytick.left"] = True      
plt.rcParams["ytick.right"] = True     
plt.rcParams["xtick.major.size"] = 8.0 
plt.rcParams["ytick.major.size"] = 8.0 
plt.rcParams["xtick.major.width"] = 1.0
plt.rcParams["ytick.major.width"] = 1.0
plt.rcParams["xtick.minor.size"] = 4.0 
plt.rcParams["ytick.minor.size"] = 4.0 
plt.rcParams["xtick.minor.width"] = 0.7
plt.rcParams["ytick.minor.width"] = 0.7
plt.rcParams["xtick.minor.visible"] = True
plt.rcParams["ytick.minor.visible"] = True
plt.rcParams["xtick.labelsize"] = 12   
plt.rcParams["ytick.labelsize"] = 12   
plt.rcParams["axes.labelsize"] = 12   
plt.rcParams["figure.facecolor"] = 'white'

In [ ]:
workDir="/home/kiyoyabe/erun/s25b/analysis"
figDir=os.path.join(workDir, "figures/fiberProfiles")
os.makedirs(figDir, exist_ok=True)

In [ ]:
run = "run24"
collections = ["PFS/calib/pipe2d-1760/run24/verifyCalib.science.20250930a"]
butler = Butler("/work/datastore", collections=collections)

spectrographs = [1, 2, 3, 4]
arms = ["b", "r", "n", "m"]
visit = 131624

## plot generated fiberProfiles

### plot fiberProfiles of a subset of fibers

In [ ]:
rows = 5
cols = 5

np.random.seed(0)

for i,spectrograph in enumerate(spectrographs):
    for j,arm in enumerate(arms):
        print(f"processing {arm}{spectrograph}")
        dataId = dict(visit=visit, spectrograph=spectrograph, arm=arm)

        fiberProfiles = butler.get("fiberProfiles", dataId)
        print(butler.getURI("fiberProfiles", dataId))

        fiberIds = list(np.random.choice(fiberProfiles.fiberId, size=rows*cols, replace=False))

        figAxes = fiberProfiles.plot(rows, cols, fontsize=8, show=False, fiberId=fiberIds)
        plt.yscale("log")
        plt.ylim(1.0e-05, 1.0e+00)
        fig,axe = figAxes[fiberIds[int(rows/2)]]
        axe.set_title(f"fiberProfiles ({arm}{spectrograph})")
        plt.savefig(os.path.join(figDir, f"fiberProfiles_{run}_profiles_{arm}{spectrograph}.pdf"), bbox_inches="tight")
        plt.savefig(os.path.join(figDir, f"fiberProfiles_{run}_profiles_{arm}{spectrograph}.png"), bbox_inches="tight", dpi=150)
        plt.close()            

## plot statistics for generated fiberProfiles

### generate a header image (with spectrograph and arm information)

In [ ]:
fig_width = 150 / 100
fig_height = 710 / 100

spectrographs = [1, 2, 3, 4]
arms = ["b", "r", "n", "m"]
for arm in arms:
    for spectrograph in spectrographs:
        fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=100)
        ax.set_facecolor('white')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        ax.text(0.5, 0.5, f'{arm}{spectrograph}', 
                fontsize=50, 
                ha='center', 
                va='center', 
                color='red',
                weight='bold')
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)

        plt.savefig(os.path.join(figDir, f'header_{arm}{spectrograph}.png'), 
                    dpi=100, 
                    bbox_inches='tight', 
                    pad_inches=0,
                    facecolor='white')
        plt.close()

### generate individual plots about statistics

In [ ]:
for i,spectrograph in enumerate(spectrographs):
    for j,arm in enumerate(arms):
        print(f"processing {arm}{spectrograph}")
        dataId = dict(visit=visit, spectrograph=spectrograph, arm=arm)
        
        fiberProfiles = butler.get("fiberProfiles", dataId)
        print(butler.getURI("fiberProfiles", dataId))

        figAxes = fiberProfiles.plotHistograms(variationRange=(0.0, 0.03), maxRange=(0.1, 0.5), show=False)
        for fig,axe in figAxes:
            filename = os.path.join(figDir, f"fiberProfiles_{run}_histogram_{axe[0].get_xlabel()}_{arm}{spectrograph}.pdf")
            plt.savefig(filename, bbox_inches="tight")
            filename = os.path.join(figDir, f"fiberProfiles_{run}_histogram_{axe[0].get_xlabel()}_{arm}{spectrograph}.png")
            plt.savefig(filename, bbox_inches="tight", dpi=150)
            plt.close()

### combine individual plots into a single plot

In [ ]:
names = ["centroid", "min", "max", "r90", "skew", "variation", "width", "wingFrac"]

grid_cols = len(names)
grid_rows = len(spectrographs)
padding = 5
bg_color=(255, 255, 255)

for arm in arms:
    print(f"process arm={arm}...")
    headers = []
    images = []
    for spectrograph in spectrographs:
        filename=os.path.join(figDir, f"header_{arm}{spectrograph}.png")
        img = Image.open(filename)        
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
        headers.append(img)
        for name in names:
            filename=os.path.join(figDir, f"fiberProfiles_{run}_histogram_{name}_{arm}{spectrograph}.png")
            try:
                img = Image.open(filename)
                if img.mode != 'RGBA':
                    img = img.convert('RGBA')
                images.append(img)
            except Exception as e:
                print(f"WARNING: reading {file_path} failed: {e}")
                images.append(None)
    target_size = images[0].size

    head_width, head_height = (150, 710)
    tile_width, tile_height = target_size
    canvas_width = head_width + grid_cols * tile_width + (grid_cols + 1) * padding
    canvas_height = grid_rows * tile_height + (grid_rows + 1) * padding
    canvas = Image.new('RGBA', (canvas_width, canvas_height), bg_color + (255,))
    
    for i in range(grid_rows):
        x = 0
        y = padding + i * (tile_height + padding)
        img = headers[i]
        canvas.paste(img, (x, y), img if img.mode == 'RGBA' else None)
        for j in range(grid_cols):
            idx = i * grid_cols + j
            x = head_width + padding + j * (tile_width + padding)
            y = padding + i * (tile_height + padding)
            if idx < len(images) and images[idx] is not None:
                img = images[idx].resize(target_size, Image.Resampling.LANCZOS)
                canvas.paste(img, (x, y), img if img.mode == 'RGBA' else None)
            else:
                placeholder = Image.new('RGBA', target_size, (240, 240, 240, 255))
                canvas.paste(placeholder, (x, y))

    canvas.save(os.path.join(figDir, f"fiberProfiles_{run}_histogram_combined_{arm}.png"), 'PNG')